In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import torch

from sbi.inference import NPE
from sbi.analysis import pairplot
from sbi.utils import BoxUniform

In [2]:
num_dim = 3

In [3]:
def simulator(theta):
    m,k,b = theta
    ts = 5.0
    dt = 0.01
    steps = int(ts/dt)
    t = np.linspace(0, ts, steps)
    xk = np.array([10,3])
    x=10
    v=3
    states = []

    for i in range(steps):
        x_new = x + dt * v
        v_new = v + dt * (-k * x / m - b * v / m) + (dt / m) * np.cos(i*dt * 10)
        x, v = x_new, v_new
        states.append([x, v])

    return torch.tensor(states).flatten()

def eval(t, x, val):
    index = np.argmin(np.abs(t - val))
    return x[index]


In [4]:
theta_o = np.array((10,5,2))
x_o = simulator(theta_o)

In [5]:
# plt.plot(t, x[:, 0], "k")

In [6]:
# plt.plot(t, x[:, 1], "k")

In [7]:
prior = BoxUniform(
    low=torch.as_tensor([1.0, 1.0, 0.5]), 
    high=torch.as_tensor([20.0, 10.0, 2.0])
)

In [8]:
theta_samples = prior.sample((1000,))
x_samples = torch.stack([simulator(theta) for theta in theta_samples])

In [9]:
# import required modules
from sbi.neural_nets import posterior_nn
from sbi.inference import SNPE

# import the different choices of pre-configured embedding networks
from sbi.neural_nets.embedding_nets import (
    FCEmbedding,
    CNNEmbedding,
    PermutationInvariantEmbedding
)
import torch.nn as nn

In [10]:
# Define a GRU-based embedding network
class TimeSeriesEmbedding(nn.Module):
    def __init__(self, input_dim, hidden_dim=32, output_dim=10):
        super().__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        _, h = self.gru(x)  # Get the last hidden state
        return self.fc(h.squeeze(0))  # Map to lower-dimensional summary

embedding_net = TimeSeriesEmbedding(input_dim=2)  # 2 inputs (x, v)

In [11]:
# instantiate the conditional neural density estimator
neural_posterior = posterior_nn(model="maf", embedding_net=embedding_net)

In [12]:
# setup the inference procedure with NPE
inference = SNPE(prior=prior, density_estimator=neural_posterior)

In [17]:
# Reshape the input data to match the expected input size
theta_samples_reshaped = theta_samples.view(-1, 3)
x_samples_reshaped = x_samples.view(-1, x_samples.size(-1))

In [18]:
theta_samples_reshaped

tensor([[13.7782,  6.8294,  0.5226],
        [15.0538,  8.0929,  1.3509],
        [12.2330,  9.7689,  0.9275],
        ...,
        [11.4373,  1.9335,  0.6238],
        [ 2.7122,  6.9144,  0.6387],
        [ 8.3824,  1.5310,  1.5553]])

In [20]:
x_samples

tensor([[ 10.0300,   2.9500,  10.0595,  ...,  -0.2003, -10.1002,  -0.1495],
        [ 10.0300,   2.9442,  10.0594,  ...,   0.8663,  -8.9253,   0.9141],
        [ 10.0300,   2.9187,  10.0592,  ...,   6.7667,  -5.1598,   6.8041],
        ...,
        [ 10.0300,   2.9823,  10.0598,  ...,  -4.5746,   2.1422,  -4.5750],
        [ 10.0300,   2.7417,  10.0574,  ...,  -9.7548,   0.8506,  -9.7526],
        [ 10.0300,   2.9774,  10.0598,  ...,  -3.7140,   2.0461,  -3.7098]])

In [ ]:
# Append simulations and train the density estimator
density_estimator = inference.append_simulations(theta_samples_reshaped, x_samples_reshaped).train()